<h2 style="text-align:center;font-size:200%;">
    <b>Workplace Accident Database Textual Analysis through LLMs</b>
</h2>
<h3  style="text-align:center;">Keywords : 
    <span style="border-radius:7px;background-color:yellowgreen;color:white;padding:7px;">Large Language Models</span>
    <span style="border-radius:7px;background-color:yellowgreen;color:white;padding:7px;">Natural Language Processing</span>
    <span style="border-radius:7px;background-color:yellowgreen;color:white;padding:7px;">Mistral</span>
    <span style="border-radius:7px;background-color:yellowgreen;color:white;padding:7px;">Work Accidents</span>
    <span style="border-radius:7px;background-color:yellowgreen;color:white;padding:7px;">EHS</span>
</h3>


The [EPICEA database](https://www.inrs.fr/publications/bdd/epicea.html) is managed by a french institute called [INRS](https://www.inrs.fr/), in charge of risk preventions in work environments.

The purpose of this notebook is to create a tool able to : 
1. extract massively the accident descriptions from the french "EPICEA" database.,
2. apply a LLM prompt in order to extract structured information from unstructured description
3. propose a first analysis of the database

In [ ]:
Epicea est une base de données nationale et anonyme rassemblant plus de 21 000 cas d'accidents du travail survenus, depuis 1990, à des salariés du régime général de la Sécurité sociale. Ces accidents sont mortels, graves ou significatifs pour la prévention.

Cette base de données n'est pas exhaustive puisque tous les accidents du travail n'y sont pas répertoriés.

In [ ]:
L'anonymat des personnes physiques et morales est respecté et l'origine des informations est préservée.

In [ ]:
Le numéro du dossier (qui s'incrémente automatiquement) : plus le numéro est élevé, plus l'accident est récent
Le comité technique national (classification des grands secteurs d'activité selon l'arrêté du 17 octobre 1995 modifié)
Le code entreprise (jusqu'en 2015 : code risque, déclinaison des comités techniques nationaux ; à partir de janvier 2015 : code APE selon la nomenclature NAF)
Le facteur matériel le plus proche des lésions : objet, matériel, matériau, installation, etc. intervenant dans l'accident
Le récit circonstancié de l'accident, éventuellement complété par des documents attachés (photos, arbres des causes, schémas, etc.)

Le facteur matériel (ou matériel en cause) est structuré et renvoie à un libellé plus ou moins détaillé. Par exemple 510210 concerne les toitures en matériaux fragiles, 5102* une partie de bâtiment ou d’ouvrage, 51* les zones géographiques et emplacements de travail.

Une collection de dossiers est obtenue par sélection multicritère.

# <div style="text-align: left; background-color: yellowgreen; color: white; padding: 10px; line-height:1;border-radius:10px">1. Modules and dependancies installing</div>

In [8]:
from tqdm import tqdm
from urllib.request import urlopen
import json
import time
import pandas as pd
from io import StringIO
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import re
from IPython.display import Markdown as md

In [2]:
# Configurer les options Chrome pour le mode sans tête
chrome_options = Options()
chrome_options.add_argument("--headless")
# Initialiser le pilote Chrome avec les options spécifiées
driver = webdriver.Chrome()#options=chrome_options)
waiting_time = 1.5

# <div style="text-align: left; background-color: yellowgreen; color: white; padding: 10px; line-height:1;border-radius:10px">2. Data Collection from INRS website</div>

Web scraping, is a technique used in data science to automatically extract data from websites.
It involves using a program or script to navigate through web pages, parse the HTML or XML code, and extract specific pieces of information, such as text, images, files or other structured data. 

Our web scraping strategy will be performed in 2 separate steps:
- First we will get the list of accident #IDs available in the database
- Then we will extract separately the informations related to each individual accident.

## 2.1. Accidents #ID extraction

In [3]:
# Ouverture du site
search_url="https://www.inrs.fr/publications/bdd/epicea/recherche.html"
driver.get(search_url)

In [4]:
# Clique sur le bouton d'acceptation des cookies
time.sleep(waiting_time)
bouton_cookie = driver.find_element(By.ID, "onetrust-accept-btn-handler")
bouton_cookie.click()

In [5]:
# Clique sur le bouton de "recherche"
time.sleep(waiting_time)
driver.switch_to.frame('siteExterneIframe')
bouton_recherche = driver.find_element(By.XPATH, '//img[@src="/EPICEA/epicea.nsf/Rechercher.jpg"]')
bouton_recherche.click()

In [6]:
# Clique sur le bouton "Afficher la liste"
time.sleep(waiting_time)
bouton_afficher = driver.find_element(By.LINK_TEXT, "afficher la liste")
bouton_afficher.click()

In [7]:
# Recherche du nombre de pages
time.sleep(waiting_time)
bouton_last_page = driver.find_element(By.LINK_TEXT, ">>")

In [8]:
nb_pages = bouton_last_page.get_attribute('href')
nb_pages = nb_pages.replace("javascript:mainForm.currentPage.value='", "")
nb_pages = nb_pages.replace("';mainForm.submit();", "")
nb_pages = int(nb_pages)

In [9]:
# for i in liens_ref:
#     print(i.get_attribute('href'))

In [10]:
# Main loop
Ref_list = []

liens_ref = driver.find_elements(By.CLASS_NAME, 'lien')
for i in liens_ref:
    if "unid" in i.get_attribute('href'):
        Ref_list.append(i.get_attribute('href'))
        
for i in tqdm(range(1, nb_pages)):
    # Clique sur le lien page suivante
    time.sleep(waiting_time)
    bouton_next_page = driver.find_element(By.LINK_TEXT, ">")
    bouton_next_page.click()
    liens_ref = driver.find_elements(By.CLASS_NAME, 'lien')
    for j in liens_ref:
        if "unid" in j.get_attribute('href'):
            Ref_list.append(j.get_attribute('href'))
    

  2%|█▋                                                                            | 95/4335 [03:37<2:41:35,  2.29s/it]


KeyboardInterrupt: 

In [ ]:
Ref_list = list(set(Ref_list))
number_id = len(Ref_list)

In [ ]:
df = pd.DataFrame(Ref_list, columns=['Ref'])

In [ ]:
df.to_csv('Ref_epicea.csv', index=False)

In [12]:
# Fermer le navigateur
driver.quit()

In [20]:
md("After this first step, we get a list of {} ID numbers, saved in the *Ref_epicea.csv* file.".format(number_id)) 

After this first step, we get a list of 5 ID numbers, saved in the *Ref_epicea.csv* file.

## 2.2. Detailed data extraction

In [9]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
from tqdm import tqdm
from ast import literal_eval
from os.path import exists
from pathlib import Path
waiting_time = 1

In [10]:
# Ouverture du site
driver = webdriver.Chrome()#options=chrome_options)
search_url="https://www.inrs.fr/publications/bdd/epicea/recherche.html"
driver.get(search_url)
# Clique sur le bouton d'acceptation des cookies
time.sleep(waiting_time)
bouton_cookie = driver.find_element(By.ID, "onetrust-accept-btn-handler")
bouton_cookie.click()
# Clique sur le bouton de "recherche"
time.sleep(waiting_time)
driver.switch_to.frame('siteExterneIframe')
bouton_recherche = driver.find_element(By.XPATH, '//img[@src="/EPICEA/epicea.nsf/Rechercher.jpg"]')
bouton_recherche.click()
# Clique sur le bouton "Afficher la liste"
time.sleep(waiting_time)
bouton_afficher = driver.find_element(By.LINK_TEXT, "afficher la liste")
bouton_afficher.click()

In [11]:
# Data loading
df = pd.read_csv('Ref_epicea.csv')

In [12]:
path = Path("Accident_database.csv")
if path.is_file():
    df_deja_analyse = pd.read_csv('Accident_database.csv', sep="|")
else:
    df_deja_analyse = pd.DataFrame(columns=['Ref', 'Numero_dossier', 'Comite', 'Code_entreprise', 'Materiel', 'Resume', 'Adresse_pdf'])

In [13]:
ref_deja_analyses = df_deja_analyse['Ref']

In [14]:
# Filtrer df1 pour garder seulement les lignes dont l'ID n'est pas dans df2
df = df[~df['Ref'].isin(ref_deja_analyses)]

In [8]:
# Dataframe columns creation
col = ['Numero_dossier', 'Comite', 'Code_entreprise', 'Materiel', 'Resume', 'Adresse_pdf']
for i in col:
    df[col] = None

In [12]:
def get_content_after_text(driver, text):
    # Remplacer les espaces par un code qui représente un espace insécable en HTML
    text_for_xpath = text.replace(' ', '&#160;')
    
    # Préparer l'expression XPath
    # Nous construisons d'abord la partie avec les espaces insécables en dehors de la f-string
    nbsp = '\xa0'
    xpath_text_part = text_for_xpath.replace('&#160;', nbsp)
    
    # Ensuite, nous l'insérons dans l'expression XPath
    xpath = f"//td[contains(., '{xpath_text_part}')]/following-sibling::td[1]"
    
    # Trouver l'élément par XPath
    content_td = driver.find_element(By.XPATH, xpath)
    
    # Retourner le texte de cet élément
    return content_td.text.strip()

In [13]:
def get_accident_summary(driver, text):
    # Échapper les espaces insécables présents dans le texte d'entrée
    nbsp = '\xa0'
    text_for_xpath = text.replace(' ', nbsp)
    
    # Construire l'XPath pour trouver le td contenant le texte
    # puis naviguer au premier td suivant qui contient le résumé
    xpath = f"//td[contains(., '{text_for_xpath}')]/following-sibling::td[1]//div"
    
    # Trouver l'élément par XPath
    summary_div = driver.find_element(By.XPATH, xpath)
    
    # Retourner le texte de cet élément, en utilisant `.text` pour récupérer tout le texte y compris celui des éléments enfants
    return summary_div.text.strip()

In [14]:
ref_avec_titre_nul = df.loc[df['Numero_dossier'].isnull(), 'Ref']
liste_ref = ref_avec_titre_nul.tolist()

In [15]:
for ref in tqdm(liste_ref):
    
    driver.get(ref)
    
    # Informations tabulaires
    Numero_dossier = get_content_after_text(driver, "Numéro du dossier : ")
    Comite = get_content_after_text(driver, "Comité technique national : ")
    Code_entreprise = get_content_after_text(driver, "Code entreprise : ")
    Materiel = get_content_after_text(driver, "Matériel en cause : ")
    Resume = get_content_after_text(driver, "Résumé de ")
    Resume = Resume.replace("/n", " ")
    Resume = Resume.replace("/r", " ")
    
    # lien vers les pdf
    liste_pdf = []
    pdf_links = driver.find_elements(By.CLASS_NAME, "lien")
    for i in pdf_links:
        if i.get_attribute('onclick') != None:
            text = i.get_attribute('onclick')
            text = text.replace("Javascript: window.open('", "")
            text = text.replace("','Documents', 'menubar=yes, status=no, scrollbars=yes, resizable=yes');", "")
            text = text.replace("javascript:window.open('public_popupAideAffichage','Aide','resizable=yes,status=yes,scrollbars=yes,menubars=yes,width=540,height=200')", "")
            if len(text) > 10:
                text = "https://epicea.inrs.fr/" + text.strip()
                liste_pdf.append(text)
    
    # Creation du dataframe
    dict_data = {
        'Ref':ref,
        'Numero_dossier':Numero_dossier, 
        'Comite':Comite, 
        'Code_entreprise':Code_entreprise, 
        'Materiel':Materiel, 
        'Resume':Resume, 
        'Adresse_pdf':liste_pdf,
    }
    dict_data_list = {key: [value] for key, value in dict_data.items()}
    df_temp = pd.DataFrame(dict_data_list)

    # Concatenation
    df_deja_analyse = pd.concat([df_deja_analyse, df_temp], axis=0, ignore_index = True)
    df_deja_analyse.to_csv('Accident_database.csv', sep='|', index=False, encoding="utf-8")

    time.sleep(waiting_time)


0it [00:00, ?it/s]


In [16]:
df_deja_analyse.shape

(21430, 7)

At this step, we have a table of 21430 lines, corresponding to the number of #IDs, and 6 rows containing textual information

In [21]:
df_deja_analyse.head()

NameError: name 'df_deja_analyse' is not defined

In [ ]:
df_deja_analyse.info()

# <div style="text-align: left; background-color: yellowgreen; color: white; padding: 10px; line-height:1;border-radius:10px">3. Extraction of data from narratives</div>

A part of the code will use prompt and variable name formulated in french. Because the data source in written in french, it is necessary, for better results, to write the prompts in french and to describe the expected output in french.

In [139]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
from tqdm import tqdm
from ast import literal_eval
from os.path import exists
from pathlib import Path

In [140]:
import sys
sys.path.append("C:/Users/arnaud/AppData/Roaming/Python/Python312/site-packages/")
sys.path.append("C:/Windows/System32/")
sys.path.append("C:/Users/Arnaud/AppData/Roaming/Python/Python312/site-packages/onnxruntime/capi/")
# import pandas as pd
from typing import Optional, Sequence, Generic, TypeVar, List
from langchain_openai import OpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field, Extra, validator
from ollama import Client
# For token counting
import openai
import pandas as pd
import json
from langchain.callbacks import get_openai_callback
from langchain_openai import ChatOpenAI
from langchain_mistralai import ChatMistralAI, MistralAIEmbeddings
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage
import time

In [141]:
from tqdm import tqdm
from urllib.request import urlopen
import json
import time
import pandas as pd
from io import StringIO
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import re
from IPython.display import Markdown as md

In [142]:
import json
import pandas as pd
from tqdm import tqdm
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_community.chat_models import ChatOllama
from pydantic import BaseModel, Field, Extra
import datetime

In [143]:
import json
import pandas as pd
from tqdm import tqdm
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_community.chat_models import ChatOllama
from pydantic import BaseModel, Field, Extra, field_validator, ConfigDict
from typing import List, Optional
from enum import Enum

In [144]:
from os.path import exists
from pathlib import Path

In [145]:
nom_modele = "mistral"
csv_name = "Accident_database_refined.csv"

path = Path(csv_name)
if path.is_file():
    df = pd.read_csv(csv_name, sep="|")  
else:
    df = pd.read_csv('Accident_database.csv', sep="|")
    col = ['Metier', 'Sexe', 'Age', 'Type_accident', 'Blessure', 'Deces', 'Circulation', 'Malaise',
          'Suicide','Machine', 'Cause', 'Zone']
    for i in col + ['Status']:
        df[i] = None
    liste_valeurs = ['241GM', '274CG', '295EC', '2110Z', '2120Z', '244CB', '244DA', '4646Z', '4773Z', '514NA', '523AB', 
                     '1073Z', '1086Z', '1089Z', '157AB', '158VB']
    df = df[df['Code_entreprise'].apply(lambda x: any(item in x for item in liste_valeurs))]
    df = df[df['Numero_dossier']!=19258]

In [146]:
ref_non_analysees = df.loc[df['Status'].isnull(), 'Numero_dossier']
liste_num = ref_non_analysees.tolist()

In [147]:
class BodyZone(str, Enum):
    TETE = "tete"
    TORSE = "torse"
    VENTRE = "ventre"
    DOS = "dos"
    BRAS = "bras"
    MAIN = "main"
    JAMBE = "jambe"
    PIED = "pied"
    POSTERIEUR = "posterieur"
    COEUR = "coeur"
    NA = "NA"

In [148]:
class Accident(BaseModel):
    Metier: str = Field(description="Métier, rôle ou fonction de la victime ayant subi l'accident.")
    Sexe: str = Field(description="Sexe (Homme ou Femme) de la victime ayant subi l'accident.")
    Age: int = Field(description="Âge de la victime ayant subi l'accident.")
    
    Type_accident: str = Field(description="Type d'accident survenu. 1 ou 2 mots maximum.")
    Blessure: str = Field(description="Descriptif médical des blessures ou symptômes. 1 ou 2 mots maximum.")
    
    Deces: bool = Field(description="La victime est mentionnée comme décédée.")
    Circulation: bool = Field(description="Accident lié à la circulation.")
    Malaise: bool = Field(description="Accident lié à un malaise type AVC, infarctus, accident cardiaque.")
    Suicide: bool = Field(description="Accident lié à un suicide.")
    
    Machine: List[str] = Field(description="Machines, pièces ou objets impliqués dans l'accident. 1 ou 2 mots maximum par élément.")
    Cause: List[str] = Field(description="Facteurs ayant causé directement ou ayant favorisé l'accident. 1 à 3 mots maximum par facteur.")
    Zone: BodyZone = Field(description="Zone du corps concernée par l'accident.")
        
    @field_validator('Sexe')
    @classmethod
    def sexe_valide(cls, v):
        if v.lower() not in ['homme', 'femme']:
            raise ValueError('Le sexe doit être "Homme" ou "Femme"')
        return v.capitalize()

    @field_validator('Age')
    @classmethod
    def age_valide(cls, v):
        if v is not None and (v < -1 or v > 120):
            raise ValueError('L\'âge doit être entre 0 et 120')
        return v

    @field_validator('Metier', 'Type_accident', 'Blessure')
    @classmethod
    def non_vide(cls, v):
        if not v.strip():
            raise ValueError('Ce champ ne peut pas être vide')
        return v

    @field_validator('Machine', 'Cause')
    @classmethod
    def liste_non_vide(cls, v):
        if not v:
            return ['Non spécifié']
        return [item.strip() for item in v if item.strip()]

    @field_validator('Zone')
    @classmethod
    def zone_valide(cls, v):
        zone_mapping = {
            'tete': ['crane', 'visage', 'cou', 'cerveau'],
            'torse': ['poitrine', 'torse', 'poumon'],
            'ventre': ['ventre', 'estomac'],
            'dos': ['dos', 'epaule'],
            'bras': ['bras', 'coude', 'epaule'],
            'main': ['main', 'doigt', 'poignet'],
            'jambe': ['genou', 'cuisse', 'mollet', 'tibia'],
            'pied': ['pied', 'cheville'],
            'posterieur': ['fesses'],
            'coeur': ['coeur']
        }
        
        v = v.lower()
        for zone, keywords in zone_mapping.items():
            if v in keywords:
                return BodyZone(zone)
        return BodyZone.NA

    model_config = ConfigDict(
        extra='forbid',
        use_enum_values=True,
        json_schema_extra={
            'examples': [
                {
                    'Metier': 'Technicien de maintenance',
                    'Sexe': 'Homme',
                    'Age': 45,
                    'Type_accident': 'Chute',
                    'Blessure': 'Fracture',
                    'Deces': False,
                    'Circulation': False,
                    'Malaise': False,
                    'Suicide': False,
                    'Machine': ['Échelle'],
                    'Cause': ['Sol glissant', 'Manque d\'EPI'],
                    'Zone': 'jambe'
                }
            ]
        }
    )

In [149]:
def standardize_metier(metier):
    if isinstance(metier, list):
        return ', '.join(metier)
    return str(metier)

In [150]:
def standardize_sex(sex):
    sex = str(sex).lower()
    sex = str(sex).replace(",", "")
    sex = str(sex).replace("é", "e")
    sex = str(sex).replace("ou", "")
    sex = str(sex).replace("or", "")
    sex = str(sex).replace(" ", "")
    if sex in ['homme', 'masculin', 'male', 'm']:
        return 'Homme'
    elif sex in ['femme', 'feminin', 'female', 'f']:
        return 'Femme'
    elif sex in ['homme'] and sex in ['femme']:
        return None
    elif 'NA':
        return None
    elif sex == '':
        return None
    else:
        return None

In [151]:
def standardize_zone(zone):
    zone_mapping = {
        'tete': ['crane', 'visage', 'cou', 'cerveau'],
        'torse': ['poitrine', 'torse', 'poumon'],
        'ventre': ['ventre', 'estomac'],
        'dos': ['dos', 'epaule'],
        'bras': ['bras', 'coude', 'epaule'],
        'main': ['main', 'doigt', 'poignet'],
        'jambe': ['genou', 'cuisse', 'mollet', 'tibia'],
        'pied': ['pied', 'cheville'],
        'posterieur': ['fesses'],
        'coeur': ['coeur']
    }

    zone = str(zone).lower()
    for standard_zone, keywords in zone_mapping.items():
        if zone in keywords or zone == standard_zone:
            return standard_zone
    return 'NA'

In [152]:
# Fonctions auxiliaires
def parse_json_safely(json_string):
    try:
        return json.loads(json_string)
    except json.JSONDecodeError as e:
        print(f"Erreur de parsing JSON : {e}")
        print(f"Contenu problématique : {json_string}")
        # Tentative de nettoyage basique
        cleaned = json_string.strip()
        if cleaned.startswith("```json"):
            cleaned = cleaned[7:]
        if cleaned.endswith("```"):
            cleaned = cleaned[:-3]
        try:
            return json.loads(cleaned)
        except json.JSONDecodeError as e2:
            print(f"Échec du nettoyage : {e2}")
            return None

In [153]:
def validate_content(content_dict):
    expected_keys = ['Metier', 'Sexe', 'Age', 'Type_accident', 'Blessure', 'Deces', 'Circulation', 'Malaise', 'Suicide', 'Machine', 'Cause']
    present_keys = [key for key in expected_keys if key in content_dict]
    missing_keys = [key for key in expected_keys if key not in content_dict]
    if missing_keys:
        print(f"Clés manquantes : {missing_keys}")
    # On considère le contenu valide s'il contient au moins 3 champs
    return len(present_keys) >= 3

In [154]:
def add_default_values(content_dict):
    default_values = {
        'Metier': None,
        'Sexe': None,
        'Age': None,
        'Type_accident': None,
        'Blessure': None,
        'Deces': None,
        'Circulation': None,
        'Malaise': None,
        'Suicide': None,
        'Machine': None,
        'Cause': None
    }
    for key, value in default_values.items():
        if key not in content_dict:
            content_dict[key] = value
    return content_dict

In [155]:
def clean_and_standardize_content(content_dict):
    
    for key, value in content_dict.items():
        if key == 'Sexe':
            content_dict[key] = standardize_sex(value)
        elif key == 'Metier':
            content_dict[key] = standardize_metier(value)
        elif key == 'Zone':
            content_dict[key] = standardize_zone(value)
        elif isinstance(value, list):
            content_dict[key] = ', '.join(map(str, value))
        elif value is None:
            content_dict[key] = 'Non spécifié'
        elif isinstance(value, bool):
            content_dict[key] = 'Oui' if value else 'Non'
        else:
            content_dict[key] = str(value)
            
    return content_dict

In [156]:
def process_content_dict(content_dict):
    """
    Traite le dictionnaire de contenu, qu'il soit déjà un dictionnaire
    ou une chaîne JSON.
    """
    if isinstance(content_dict, dict):
        # Le contenu est déjà un dictionnaire, pas besoin de le parser
        return content_dict
    elif isinstance(content_dict, str):
        # Le contenu est une chaîne JSON, on la parse
        return json.loads(content_dict)
    else:
        # Type inattendu, on lève une exception
        raise TypeError(f"Type inattendu pour content_dict: {type(content_dict)}")

In [157]:
# Configuration du modèle et du prompt
pydantic_parser = PydanticOutputParser(pydantic_object=Accident)
format_instructions = pydantic_parser.get_format_instructions()

In [158]:
template_string = """Tu es un analyste français qui relit des compte-rendu d'accidents et effectue des saisies. 
Analyse le texte ci-dessous qui se trouve entre les triples apostrophes et extrais-en les informations requises. 

Descriptif de l'accident : ```{descriptif}```

IMPORTANT:
- Toutes tes réponses DOIVENT être en français.
- Pour le champ 'Sexe', utilise UNIQUEMENT 'Homme' ou 'Femme'.
- Le champ 'Metier' doit être une chaîne de caractères, et pas une liste.
- Pour le champ 'Zone', utilise UNIQUEMENT l'une des valeurs suivantes selon la zone du corps concernée :
  - 'tete' pour [crane, visage, cou, cerveau]
  - 'torse' pour [poitrine, torse, poumon]
  - 'ventre' pour [ventre, estomac]
  - 'dos' pour [dos, epaule]
  - 'bras' pour [bras, coude, epaule]
  - 'main' pour [main, doigt, poignet]
  - 'jambe' pour [genou, cuisse, mollet, tibia]
  - 'pied' pour [pied, cheville]
  - 'posterieur' pour [fesses]
  - 'coeur' pour [coeur]
  - 'NA' si l'information n'est pas présente
- Si l'information n'apparait pas dans le narratif, utilise 'NA' pour les champs textuels, 

Ta réponse DOIT être un objet JSON valide, respectant strictement le schéma suivant. N'inclus AUCUN texte en dehors de cet objet JSON.

{format_instructions}
"""

In [159]:
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template_string)  
    ],
    input_variables=["descriptif"],
    partial_variables={"format_instructions": format_instructions}
)

In [160]:
# Configuration du modèle Ollama
llm = ChatOllama(
    model=nom_modele, 
    format="json",
    temperature=0,
    top_k=10,
    top_p=0.9,
    repeat_penalty=1.1
)

In [ ]:
start_time = time.time()

# Boucle principale
for num in tqdm(liste_num[7:]):
    descriptif = df.loc[df['Numero_dossier'] == num, 'Resume'].item()
    messages = prompt.format_messages(descriptif=descriptif)

    chat_model_response = llm.invoke(messages)
    print(f"Réponse brute pour le numéro {num}:")
    print(datetime.datetime.now())
    print(chat_model_response.content)

    content_dict = parse_json_safely(chat_model_response.content)

    if content_dict:
        content_dict = process_content_dict(content_dict)
        content_dict = add_default_values(content_dict)
        content_dict = clean_and_standardize_content(content_dict)

        if validate_content(content_dict):
            for j in content_dict.keys():
                df.loc[df['Numero_dossier'] == num, j] = content_dict[j]
            df.loc[df['Numero_dossier'] == num, 'Status'] = 'Analysé'
        else:
            print(f"Avertissement : Résultat incomplet pour le numéro {num}")
            print(f"Contenu du dictionnaire : {content_dict}")
    else:
        print(f"Erreur : Impossible de parser le résultat pour le numéro {num}")
        print(f"Descriptif pour le numéro {num}:")
        print(descriptif)
        continue

    # Sauvegarde incrémentale
    nom_csv = 'Accident_database_refined.csv'
    df.to_csv(nom_csv, sep='|', index=False, encoding="utf-8")
    
        
inference_time = "--- %s seconds ---" % (time.time() - start_time)
print("Analyse terminée.")
print(inference_time)

  0%|                                                                                           | 0/25 [00:00<?, ?it/s]C:\Users\Arnaud\AppData\Local\Temp\ipykernel_10740\764651543.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '45' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['Numero_dossier'] == num, j] = content_dict[j]
  4%|███▎                                                                               | 1/25 [01:24<33:36, 84.02s/it]

Réponse brute pour le numéro 25950:
2024-09-06 18:43:26.707663
{"Metier": "opératrice fabrication",
"Sexe": "Femme",
"Age": 45,
"Type_accident": "Percussion",
"Blessure": "Écrasement de la cage thoracique",
"Deces": true,
"Circulation": false,
"Malaise": false,
"Suicide": false,
"Machine": ["chariot automatique"],
"Cause": [
"Maintenance en cours",
"Dépannage en cours"],
"Zone": "torse"}


  8%|██████▋                                                                            | 2/25 [02:38<30:02, 78.36s/it]

Réponse brute pour le numéro 21603:
2024-09-06 18:44:41.102160
{"Metier": "déléguée médicale généraliste", "Sexe": "Femme", "Age": 25, "Type_accident": "Accident de voiture", "Blessure": "Décédée", "Deces": true, "Circulation": "NA", "Malaise": "NA", "Suicide": "NA", "Machine": ["poids lourd"], "Cause": ["suivre un poids lourd"], "Zone": "tete"}


 12%|█████████▉                                                                         | 3/25 [04:12<31:26, 85.75s/it]

...

Réponse brute pour le numéro 6600:
2024-09-06 19:01:59.440030
{"Metier": "Technicien de bureau d'études", "Sexe": "Femme", "Age": 45, "Type_accident": "Malaise", "Blessure": "NA", "Deces": true, "Circulation": "NA", "Malaise": false, "Suicide": false, "Machine": "NA", "Cause": ["Examen médical"], "Zone": "tete"}


 64%|████████████████████████████████████████████████████▍                             | 16/25 [21:11<11:29, 76.61s/it]

Réponse brute pour le numéro 17217:
2024-09-06 19:03:14.576037
{"Metier": "agent de maîtrise entretien",
"Sexe": "Homme",
"Age": 43,
"Type_accident": "Chute",
"Blessure": "Décédé",
"Deces": true,
"Circulation": false,
"Malaise": false,
"Suicide": false,
"Machine": ["entreprise contiguë"],
"Cause": ["explosion"],
"Zone": "tete"}


 68%|███████████████████████████████████████████████████████▊                          | 17/25 [22:24<10:03, 75.44s/it]

Réponse brute pour le numéro 12850:
2024-09-06 19:04:27.305738
{"Metier": "fabrication de produits pharmaceutiques", "Sexe": "Femme", "Age": 26, "Type_accident": "Accident d'avion", "Blessure": "Décédée", "Deces": true, "Circulation": false, "Malaise": false, "Suicide": false, "Machine": ["avion"], "Cause": ["NA"], "Zone": "NA"}


In [73]:
df = pd.read_csv("Accident_database_refined.csv", sep="|")
df.head()

,Ref,Numero_dossier,Comite,Code_entreprise,Materiel,Resume,Adresse_pdf,Metier,Sexe,Age,Junior,Type_accident,Blessure,Deces,Circulation,Malaise,Suicide,Machine,Cause,Status
0,https://epicea.inrs.fr/servlet/public_display?...,20057,"E - Chimie, caoutchouc, plasturgie",244CB - Fabrication de produits de base pour l...,"110190 - Autre type de machine à remplir, emba...","Le rôle de la victime - 45 ans, conductrice de...",[],conductrice,Femme,45.0,False,défaut de machine,sectionné,False,False,False,False,machine à conditionner,dysfonctionnement,Analysé
1,https://epicea.inrs.fr/servlet/public_display?...,12123,"E - Chimie, caoutchouc, plasturgie",244CB - Fabrication de produits de base pour l...,"279000 - Autre matériel de production, de tran...","La victime, dessinateur, âgée de 29 ans, était...",[],dessinateur,Femme ou Homme,29.0,False,essai d'étanchéité,5 blessés,False,True,False,False,réacteur,canalisation s'est dessartie,Analysé
2,https://epicea.inrs.fr/servlet/public_display?...,17938,A - Métallurgie,295EC - Fabrication de machines pour les indus...,998000 - Sans objet,"La victime, 49 ans, technicien service après v...",[],technicien service après vente,Femme,49.0,False,agression,coups à la tête et au corps,True,False,False,False,Non spécifié,Non spécifié,Analysé
3,https://epicea.inrs.fr/servlet/public_display?...,6208,"D - Services, commerces et industries de l'ali...",158VB - Industries alimentaires non classées p...,110127 - Machine à coller les étiquettes,"Lors de l'étiquetage, des étiquettes se sont c...",[],magasinier,Homme,31.0,False,étiquetage,main,False,False,False,False,machine à étiqueter,"étiquettes coincées, portes ouvertes",Analysé
4,https://epicea.inrs.fr/servlet/public_display?...,8185,"D - Services, commerces et industries de l'ali...","157AB - Malterie, meunerie, fabrication de pât...",130159 - Conteneur,"La victime - manutentionnaire, âgé de 59 ans -...",[],manutentionnaire,Homme,59.0,False,chute,"clavicule, thorax, bassin",False,False,False,False,"gerbeur, camion","renversement du container, décolmatage de produit",Analysé


In [ ]:
df.info()

In [ ]:
df = df.drop([],axis=1)
# df = df.replace("\u00e9", "é")

In [ ]:
df.to_csv("Accident_database_refined.csv", sep="|", index=False, encoding="utf-8")

In [78]:
code = ['241GM', '274CG', '295EC', '2110Z', '2120Z', '244CB', '244DA', '4646Z', '4773Z', '514NA', '523AB', 
        '1073Z', '1086Z', '1089Z', '157AB', '158VB']
descriptif = ['Chimie organique de synthèse', 'Electrométallurgie et électrochimie associées', 'Fabrication de machines pour les industries du process',
             'Fabrication de produits pharmaceutiques de base', 'Fabrication de préparations pharmaceutiques', 'Fabrication de produits de base pour la pharmacie',
             'Fabrication d autres produits pharmaceutiques', 'Commerce de gros inter-entreprise de produits pharmaceutiques', 'Commerce de détail de produits pharmaceutiques', 
             'Commerce de gros de produits pharmaceutiques', 'Commerce de parfums et produits de beauté', 'Fabrication de pâtes alimentaires',
             'Fabrication d aliments diététiques', 'Fabrication d autres produits alimentaires', 'Malterie, meunerie, fabrication de pâtes alimentaires et semoule',
             'Industries alimentaires non classées']

filtres = pd.DataFrame(list(zip(code, descriptif)), columns=['Code', 'Descriptif'])
filtres.head(15)

,Code,Descriptif
0,241GM,Chimie organique de synthèse
1,274CG,Electrométallurgie et électrochimie associées
2,295EC,Fabrication de machines pour les industries du...
3,2110Z,Fabrication de produits pharmaceutiques de base
4,2120Z,Fabrication de préparations pharmaceutiques
5,244CB,Fabrication de produits de base pour la pharmacie
6,244DA,Fabrication d autres produits pharmaceutiques
7,4646Z,Commerce de gros inter-entreprise de produits ...
8,4773Z,Commerce de détail de produits pharmaceutiques
9,514NA,Commerce de gros de produits pharmaceutiques
